## 使用自适应集成(adaBoost)监督分类器，对Sentinel-2A影像进行地表覆盖分类

训练样本的标签类别使用 <a href="https://engine-aiearth.aliyun.com/#/dataset/ESA_WORLD_COVER_V100" target="_blank">ESA地表覆盖分类</a>，开发者也可自行上传训练样本。同时需注意：训练样本过多将会增加训练耗时。

### 初始化环境

In [1]:
import aie

aie.Authenticate()
aie.Initialize()

### 研究区及标签类别

用指定研究区裁剪ESA地表覆盖分类影像，同时将分类值映射到[0, 10]连续区间。需注意：分类标签必须是数值类型，同时不能小于0。

In [2]:
# 指定研究区域
roi = aie.Geometry.BBox(119.8679, 29.7058, 121.0332, 30.7178)

# 使用ESA地表覆盖分类作为标签类别, 并将其裁剪到研究区
lc = aie.ImageCollection('ESA_WORLD_COVER_V100').filterBounds(roi).mosaic().clip(roi)

# 将ESA地表覆盖分类值重新映射到[0, 10]
class_values = [10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 100]
remap_values = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
label = 'lc'
lc_remap = lc.remap(class_values, remap_values).rename([label])

### 获取样本

在研究区内生成随机样本点，Sentinel-2A影像辐射波段(以B开头的波段)像元值作为训练特征，ESA地表覆盖分类波段像元值作为类别标签。需注意：如果样本获取范围设置过大，调用sample函数将会比较耗时。本案例只提供在线生成样本点的一种参考方式，开发者可自行上传训练样本。

In [3]:
# 选取一景S2A影像的地表辐射波段, 作为训练和预测示例
img = aie.Image('S2A_MSIL2A_20220103T024121_N0301_R089_T51RTP_20220103T050347').select('B.*')

# 指定获取样本的范围, 如果样本范围过大, 同时采样分辨率过高的话, 将导致sample函数运行时间过长
sample_roi = aie.Geometry.BBox(120.1730, 30.1086, 120.3846, 30.2862)
# 指定在10m分辨率下获取1000个样本点, 由于sample函数无法保证各类别样本数量均衡, 因此可自行上传训练样本
samples = img.addBands(lc_remap).sample(sample_roi, 10, 1000)

### 划分样本

在训练样本中增加一列随机数，基于随机数将样本划分为训练样本和验证样本。

In [4]:
# 在训练样本中增加一列随机数, 选取80%的样本为训练样本, 选取20%的样本为验证样本
sample = samples.randomColumn()
training_sample = sample.filter(aie.Filter.lte('random', 0.8))
validation_sample = sample.filter(aie.Filter.gt('random', 0.8))

### 创建监督分类器

设定分类器类型，平台已提供7种典型监督分类器，开发者可按需使用。同时每种分类器所需参数有所区别，请自行设定。本案例选取自适应集成分类器，同时设置训练所需特征名称和标签名称。需注意：训练样本集每个样本必须包含指定特征和标签。

In [5]:
# 创建自适应集成分类器，并进行训练
trained_classifier = aie.Classifier.adaBoost(10).train(training_sample, label, img.bandNames().getInfo())

### 训练评估

获取训练样本的混淆矩阵, 并对训练精度进行评估。由于选取的训练区没有包含ESA地表分类所有类型，所以生成的混淆矩阵只包含部分类别。

In [6]:
# 需注意，由于试验区采集到的样本只包含部分类别，因此混淆矩阵的维度并不与remap_values对应
# 训练集混淆矩阵包含的类别，对应class_values中的类别为[10, 30, 40, 50, 60, 70]
train_accuracy = trained_classifier.confusionMatrix()
print('Training error matrix:', train_accuracy.getInfo())
print('Training overall accuracy:', train_accuracy.accuracy().getInfo())

# 使用验证集对分类器进行评估
validation = validation_sample.classify(trained_classifier)
# 验证集混淆矩阵包含的类别，对应class_values中的类别为[10, 40, 50, 60, 70]
validation_accuracy = validation.errorMatrix(label, 'classification')
print('Validation error matrix:', validation_accuracy.getInfo())
print('Validation accuracy:', validation_accuracy.accuracy().getInfo())

### 预测Sentinel-2A影像

对监督分类结果、ESA参考分类和原始影像进行可视化展示。需注意：监督分类结果的好坏主要取决于训练样本质量，以及分类器的选择。由于ESA参考分类结果是公开数据，其具备较好的分类精度。因此，本示例并不承诺自适应集成分类器结果优于ESA结果，本示例只提供API函数调用方式，请开发者酌情使用。需注意：待预测影像的波段名称必须与训练分类器时所用的训练特征名称相同，如果不一致，可使用 *<a href="https://engine-aiearth.aliyun.com/docs/page/api?d=1263e1" target="_blank">rename</a>* 函数对影像波段进行重命名。

In [7]:
# 使用训练好的分类器对影像进行分类
img_classified = img.classify(trained_classifier)

color = ['#006400' ,'#ffbb22', '#ffff4c', '#f096ff', '#fa0000', '#b4b4b4',
         '#f0f0f0', '#0064c8', '#0096a0', '#00cf75', '#fae6a0']

map = aie.Map(
    center=roi.getCenter(),
    height=800,
    zoom=9
)

# 待分类影像真彩色图层
map.addLayer(
    img,
    {
    'bands': ['B4', 'B3', 'B2'],
    'min': 200,
    'max': 1800
    },
    'True Color (432)',
    bounds=roi.getBounds()
)

# ESA地表覆盖分类图层
map.addLayer(
    lc_remap,
    {
    'bands': 'lc',
    'min': 0,
    'max': 10,
    'palette': color
    },
    'esa_classification',
    bounds=roi.getBounds()
)

# 自适应集成分类图层
map.addLayer(
    img_classified,
    {
    'bands': 'classification',
    'min': 0,
    'max': 10,
    'palette': color
    },
    'aie_classification',
    bounds=roi.getBounds()
)
map

Map(center=[30.2118, 120.45054999999999], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_…